In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime, timedelta, timezone
import pytz
import scipy.stats
from dateutil.relativedelta import relativedelta

import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

today = datetime.utcnow()

from api_keys import espn_link

x = 10

In [2]:
def summary_statistics(df):
    temp_build = df.groupby('playerId').agg({
        'name': 'first',
        'team': 'last',
        'position': 'first',
        'toi': 'sum',
        'gamesPlayed': 'sum',
        'goals': 'sum',
        'assists': 'sum',
        'specialTeams': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
        'powerPlayToi': 'sum',
        'shorthandedToi': 'sum',
        'plusMinus': 'sum',
        'fantasyPoints': 'sum',
        'secondaryPosition': 'first',
        'tertiaryPosition': 'first'
    }).reset_index()



    temp_build['FPP60'] = (temp_build['fantasyPoints'] / temp_build['toi'] * 3600).round(2)
    temp_build['FPPG'] = (temp_build['fantasyPoints'] / temp_build['gamesPlayed']).round(2)

    temp_build = temp_build.sort_values(by='FPPG', ascending=False)
    
    return temp_build

def summary_statistics_g(df):
    temp_build = df.groupby('playerId').agg({
        'name': 'first',
        'team': 'last',
        'position': 'first',
        'toi': 'sum',
        'gamesPlayed': 'sum',
        'saves': 'sum',
        'shots': 'sum',
        'start': 'sum',
        'shutout': 'sum',
        'fantasyPoints': 'sum'
    }).reset_index()
    
    temp_build['FPP60'] = (temp_build['fantasyPoints'] / temp_build['toi'] * 3600).round(2)
    temp_build['FPPG'] = (temp_build['fantasyPoints'] / temp_build['gamesPlayed']).round(2)
    
    temp_build = temp_build.sort_values(by='FPPG', ascending=False)
    
    return temp_build

In [4]:
homeOnlySked = pd.read_csv('data/sked.csv', index_col=False)
homeOnlySked

completeSked = pd.read_csv('data/sked_full.csv', index_col=False)
completeSked

team_names = pd.read_csv('data/team_names.csv', index_col=False)
team_names

file_name = f"data/allG_df_fp-{today.strftime('%Y-%m-%d')}.csv"
allG_df = pd.read_csv(file_name, index_col=False)
allG_df

file_name = f"data/all_df_fp-{today.strftime('%Y-%m-%d')}.csv"
all_df = pd.read_csv(file_name, index_col=False)
all_df

team_names_only = team_names[['abbreviation', 'name']]

all_df_forwards =  all_df.loc[all_df['position'] != 'D']
all_df_defense =  all_df.loc[all_df['position'] == 'D']

all_df_centers =  all_df.loc[all_df['secondaryPosition'] == 'C']
all_df_wingers =  all_df.loc[all_df['secondaryPosition'] == 'W']

file_name = f"data/summary_stats-{today.strftime('%Y-%m-%d')}.csv"
player_bios = pd.read_csv(file_name, index_col=False)

# file_name = f"data/rankings_worksheet_{(today-timedelta(days=6)).strftime('%Y-%m-%d')}.xlsx"
# # file_name = f"data/manual-adjustments.xlsx"
# rankings = pd.read_excel(file_name, index_col=False)

# fileName = f"data/summary_stats_last{x}-{today.strftime('%Y-%m-%d')}.csv"
# last_10 = pd.read_csv(fileName, index_col=False)

In [5]:
def opposition_stats(df):
    
    opponent_stats = df.groupby('opponent').agg({
        'toi': 'sum',
        'assists': 'sum',
        'goals': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
    #     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
        'fantasyPoints': 'sum',
        'gameDate': 'nunique',
        'powerPlayPoints': 'sum'
        # Add more columns as needed
    })

    opponent_stats['FPAPG'] = round(opponent_stats['fantasyPoints'] / opponent_stats['gameDate'], 2)
    opponent_stats['FPAPG_Rank'] = opponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

    # opponent_stats['BPG'] = round(opponent_stats['blocked'] / opponent_stats['date'], 2)
    # opponent_stats['BPG_Rank'] = opponent_stats['BPG'].rank(ascending=True, method='min').astype(int)

    opponent_stats = opponent_stats.rename_axis('abbreviation')

    opponent_stats = pd.merge(opponent_stats, team_names_only, on='abbreviation')

    # opponent_stats['powerPlayPoints'] = opponent_stats['powerPlayGoals'] + opponent_stats['powerPlayAssists']

    # opponent_stats = opponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists'])

    return opponent_stats

In [7]:
opp_all = opposition_stats(all_df)
opp_forward = opposition_stats(all_df_forwards)
opp_defense = opposition_stats(all_df_defense)
opp_centers = opposition_stats(all_df_centers)
opp_wingers = opposition_stats(all_df_wingers)
opp_all

,abbreviation,toi,assists,goals,shots,hits,blockedShots,fantasyPoints,gameDate,powerPlayPoints,FPAPG,FPAPG_Rank,name
0,ANA,442516,159,87,793,362,346,654.0,25,65,26.16,10,Anaheim Ducks
1,ARI,427034,117,68,768,478,317,562.1,24,46,23.42,25,Arizona Coyotes
2,BOS,428385,101,59,764,489,366,544.3,24,30,22.68,30,Boston Bruins
3,BUF,461559,149,90,775,510,355,661.0,26,44,25.42,17,Buffalo Sabres
4,CAR,443770,136,83,625,384,503,685.9,25,50,27.44,5,Carolina Hurricanes
5,CBJ,480338,150,91,911,352,460,705.8,27,30,26.14,11,Columbus Blue Jackets
6,CGY,445627,138,84,718,378,425,651.1,25,37,26.04,12,Calgary Flames
7,CHI,426510,156,88,785,411,349,661.1,24,60,27.55,4,Chicago Blackhawks
8,COL,442698,117,72,731,395,414,606.6,25,42,24.26,21,Colorado Avalanche
9,DAL,428012,115,70,756,432,390,586.8,24,29,24.45,19,Dallas Stars


In [8]:
gopponent_stats = allG_df.groupby('opponent').agg({
    'toi': 'sum',
    'goalsAgainst': 'sum',
    'shots': 'sum',
#     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
    'fantasyPoints': 'sum',
    'gameDate': 'nunique',
    'evSaves': 'sum',
    'evShots': 'sum',
    'shutout': 'sum',
    'ppSaves': 'sum',
    'ppShots': 'sum'
    # Add more columns as needed
})

gopponent_stats['FPAPG'] = round(gopponent_stats['fantasyPoints'] / gopponent_stats['gameDate'], 2)
gopponent_stats['FPAPG_Rank'] = gopponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

# gopponent_stats['SPG'] = round(gopponent_stats['shots'] / gopponent_stats['date'], 2)
# gopponent_stats['SPG_Rank'] = gopponent_stats['SPG'].rank(ascending=False, method='min').astype(int)

gopponent_stats = gopponent_stats.rename_axis('abbreviation')

gopponent_stats = pd.merge(gopponent_stats, team_names_only, on='abbreviation')

# gopponent_stats['powerPlayPoints'] = gopponent_stats['powerPlayGoals'] + gopponent_stats['powerPlayAssists']

# gopponent_stats = gopponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists', 'id'])

gopponent_stats

,abbreviation,toi,goalsAgainst,shots,fantasyPoints,gameDate,evSaves,evShots,shutout,ppSaves,ppShots,FPAPG,FPAPG_Rank,name
0,ANA,90160,65,737,6.840000e+01,25,547,591,0,98,116,2.74,9,Anaheim Ducks
1,ARI,86540,77,649,1.040000e+01,24,463,516,1,99,121,0.43,28,Arizona Coyotes
2,BOS,85606,76,759,1.560000e+01,24,541,599,0,115,133,0.65,27,Boston Bruins
3,BUF,93279,69,753,6.680000e+01,26,566,622,1,105,117,2.57,10,Buffalo Sabres
4,CAR,90483,79,866,5.040000e+01,25,658,715,1,105,122,2.02,15,Carolina Hurricanes
5,CBJ,97089,74,805,8.120000e+01,27,597,660,2,115,126,3.01,6,Columbus Blue Jackets
6,CGY,90747,67,789,7.640000e+01,25,606,660,1,99,109,3.06,5,Calgary Flames
7,CHI,86064,56,659,8.460000e+01,24,503,551,2,76,84,3.53,3,Chicago Blackhawks
8,COL,89833,81,787,2.520000e+01,25,560,617,3,127,146,1.01,24,Colorado Avalanche
9,DAL,86452,78,711,1.960000e+01,24,521,578,2,91,108,0.82,26,Dallas Stars


In [9]:
remainSked = homeOnlySked.loc[homeOnlySked['gameDT'] >= str(datetime.utcnow())]
remainSked.sort_values('gameDate', inplace=True)
remainSked

,gameID,gameDate,gameTime,awayTeam,homeTeam,gameDT,winningGoalie
709,2023020396,2023-12-07,Thursday 08:00 PM,DAL,WSH,2023-12-08 01:00:00,0
1163,2023020395,2023-12-07,Thursday 07:00 PM,TOR,OTT,2023-12-08 00:00:00,0
134,2023020402,2023-12-07,Thursday 09:00 PM,WPG,COL,2023-12-08 02:00:00,0
217,2023020404,2023-12-07,Thursday 10:00 PM,MIN,VAN,2023-12-08 03:00:00,0
1036,2023020403,2023-12-07,Thursday 10:30 PM,NJD,SEA,2023-12-08 03:30:00,0
...,...,...,...,...,...,...,...
409,2023021308,2024-04-18,Thursday 08:00 PM,VAN,WPG,2024-04-19 00:00:00,0
942,2023021309,2024-04-18,Thursday 09:00 PM,SJS,CGY,2024-04-19 01:00:00,0
40,2023021311,2024-04-18,Thursday 10:00 PM,ANA,VGK,2024-04-19 02:00:00,0
983,2023021307,2024-04-18,Thursday 07:00 PM,SEA,MIN,2024-04-18 23:00:00,0


In [10]:
team_names_FPAPG = team_names.copy()
team_names_FPAPG['G_FPAPG'] = 0
team_names_FPAPG['F_FPAPG'] = 0
team_names_FPAPG['D_FPAPG'] = 0

for index, row in remainSked.iterrows():
    index_of_away = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['awayTeam']].index[0]
    index_of_home = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['homeTeam']].index[0]
    
    tempAwayG = gopponent_stats.loc[gopponent_stats['abbreviation'] == row['homeTeam']]['FPAPG'].iloc[0]
    tempHomeG = gopponent_stats.loc[gopponent_stats['abbreviation'] == row['awayTeam']]['FPAPG'].iloc[0]
    team_names_FPAPG.at[index_of_away, 'G_FPAPG'] += tempAwayG
    team_names_FPAPG.at[index_of_home, 'G_FPAPG'] += tempHomeG
    
    tempAwayF = opp_forward.loc[opp_forward['abbreviation'] == row['homeTeam']]['FPAPG'].iloc[0]
    tempHomeF = opp_forward.loc[opp_forward['abbreviation'] == row['awayTeam']]['FPAPG'].iloc[0]
    team_names_FPAPG.at[index_of_away, 'F_FPAPG'] += tempAwayF
    team_names_FPAPG.at[index_of_home, 'F_FPAPG'] += tempHomeF
    
    tempAwayD = opp_defense.loc[opp_defense['abbreviation'] == row['homeTeam']]['FPAPG'].iloc[0]
    tempHomeD = opp_defense.loc[opp_defense['abbreviation'] == row['awayTeam']]['FPAPG'].iloc[0]
    team_names_FPAPG.at[index_of_away, 'D_FPAPG'] += tempAwayD
    team_names_FPAPG.at[index_of_home, 'D_FPAPG'] += tempHomeD
    
# Gmean = (team_names_FPAPG['G_FPAPG']/team_names_FPAPG['gamesRemaining']).mean()
# Fmean = (team_names_FPAPG['F_FPAPG']/team_names_FPAPG['gamesRemaining']).mean()
# Dmean = (team_names_FPAPG['D_FPAPG']/team_names_FPAPG['gamesRemaining']).mean()
# Gstd = (team_names_FPAPG['G_FPAPG']/team_names_FPAPG['gamesRemaining']).std()
# Fstd = (team_names_FPAPG['F_FPAPG']/team_names_FPAPG['gamesRemaining']).std()
# Dstd = (team_names_FPAPG['D_FPAPG']/team_names_FPAPG['gamesRemaining']).std()

# for index, row in team_names_FPAPG.iterrows():
#     Gmodifier = (row['G_FPAPG']/row['gamesRemaining']) - Gmean
#     adjModifier = (Gmodifier - Gmean) / Gstd
    
#     team_names_FPAPG.at[index, 'Gsos'] = adjModifier
    
#     Fmodifier = (row['F_FPAPG']/row['gamesRemaining']) - Fmean
#     adjModifier = (Fmodifier - Fmean) / Fstd
    
#     team_names_FPAPG.at[index, 'Fsos'] = adjModifier
    
#     Dmodifier = (row['D_FPAPG']/row['gamesRemaining']) - Dmean
#     adjModifier = (Dmodifier - Dmean) / Dstd
    
#     team_names_FPAPG.at[index, 'Dsos'] = adjModifier
    
team_names_FPAPG['G_FPAPG_PG'] = team_names_FPAPG['G_FPAPG']/team_names_FPAPG['gamesRemaining']
team_names_FPAPG['F_FPAPG_PG'] = team_names_FPAPG['F_FPAPG']/team_names_FPAPG['gamesRemaining']
team_names_FPAPG['D_FPAPG_PG'] = team_names_FPAPG['D_FPAPG']/team_names_FPAPG['gamesRemaining']
team_names_FPAPG['Gmod'] = team_names_FPAPG['G_FPAPG_PG'] / team_names_FPAPG['G_FPAPG_PG'].mean()
team_names_FPAPG['Fmod'] = team_names_FPAPG['F_FPAPG_PG'] / team_names_FPAPG['F_FPAPG_PG'].mean()
team_names_FPAPG['Dmod'] = team_names_FPAPG['D_FPAPG_PG'] / team_names_FPAPG['D_FPAPG_PG'].mean()
team_names_FPAPG['MOD'] = (team_names_FPAPG['Gmod'] + team_names_FPAPG['Fmod'] + team_names_FPAPG['Dmod'])/3
# team_names_FPAPG['GmodNorm'] = (team_names_FPAPG['Gmod'] - team_names_FPAPG['Gmod'].min()) / (team_names_FPAPG['Gmod'].max() - team_names_FPAPG['Gmod'].min())

team_names_FPAPG

,abbreviation,name,gamesPlayed,gamesRemaining,goalsFor,goalsAgainst,logo,conference,division,G_FPAPG,F_FPAPG,D_FPAPG,G_FPAPG_PG,F_FPAPG_PG,D_FPAPG_PG,Gmod,Fmod,Dmod
0,VGK,Vegas Golden Knights,27,55,89,64,https://assets.nhle.com/logos/nhl/svg/VGK_ligh...,Western,Pacific,94.86,894.03,514.45,1.724727,16.255091,9.353636,0.940814,1.007705,1.025361
1,NYR,New York Rangers,24,58,82,66,https://assets.nhle.com/logos/nhl/svg/NYR_ligh...,Eastern,Metropolitan,117.05,947.93,522.65,2.018103,16.343621,9.011207,1.100846,1.013193,0.987824
2,BOS,Boston Bruins,24,58,81,59,https://assets.nhle.com/logos/nhl/svg/BOS_ligh...,Eastern,Atlantic,105.80,940.78,523.65,1.824138,16.220345,9.028448,0.995041,1.005551,0.989714
3,COL,Colorado Avalanche,25,57,90,73,https://assets.nhle.com/logos/nhl/svg/COL_ligh...,Western,Central,101.46,914.33,518.98,1.780000,16.040877,9.104912,0.970964,0.994425,0.998096
4,LAK,Los Angeles Kings,22,60,85,53,https://assets.nhle.com/logos/nhl/svg/LAK_ligh...,Western,Pacific,117.75,983.38,562.58,1.962500,16.389667,9.376333,1.070515,1.016048,1.027849
5,VAN,Vancouver Canucks,26,56,101,70,https://assets.nhle.com/logos/nhl/svg/VAN_ligh...,Western,Pacific,102.12,887.30,506.81,1.823571,15.844643,9.050179,0.994732,0.982260,0.992096
6,FLA,Florida Panthers,25,57,77,66,https://assets.nhle.com/logos/nhl/svg/FLA_ligh...,Eastern,Atlantic,98.30,916.19,509.38,1.724561,16.073509,8.936491,0.940723,0.996448,0.979633
7,DET,Detroit Red Wings,24,58,91,71,https://assets.nhle.com/logos/nhl/svg/DET_ligh...,Eastern,Atlantic,113.29,939.09,524.83,1.953276,16.191207,9.048793,1.065484,1.003744,0.991944
8,DAL,Dallas Stars,24,58,83,71,https://assets.nhle.com/logos/nhl/svg/DAL_ligh...,Western,Central,109.34,943.95,532.03,1.885172,16.275000,9.172931,1.028334,1.008939,1.005552
9,WPG,Winnipeg Jets,24,58,77,66,https://assets.nhle.com/logos/nhl/svg/WPG_ligh...,Western,Central,112.29,940.19,536.18,1.936034,16.210172,9.244483,1.056079,1.004920,1.013396


In [ ]:
all_df